# MCT4052 Workshop 7b: Posture Mapping to Real Valued Parameters

*Author: Stefano Fasciani, stefano.fasciani@imv.uio.no, Department of Musicology, University of Oslo.*

This notebook uses the same data (and postures) presented in Workshop 5a, but here we train a regressor instead of a classifier. In particular we selected a MLP regressor that output two real-valued quantities with the range [0,1] which we can later use to control any music-related continuous value parameter (for example we can further forward them via OSC). With an MLP regressor we can chose to hany any number of output real-valued parameters (if too many with few input features, these may result highly correlated and poorly independently controllable).

The code in this notebook is similar to Workshop 5a, but instead of creating a labels we create a bidimensional array of arbitrary target values for the controld signals (associated with the postures). These are meant to be learned by the regressor. Note that the data not associated with the four postures is discarded as is not needed (there is no 'other' in this context). The regressor will 'decide' how to interpolate across postures (and associated output values) then at the input there is data that does not belong to any of the four postures.

In terms of sysem output there s a major difference as the regressor is capable to interpolate the output values across the postures, and therefore we achieve continuous control of parameters through gestures.

Also this example requires [oscHook](https://play.google.com/store/apps/details?id=com.hollyhook.oscHook&hl=en&gl=US) on Android, and for back compatibility reasons, in the oscHook v2 application we have set 'Data range' and 'Output range' to [-10;10] for both accelerometer and linear acceleration. If you are using oscHook v1 you have to rename 'accelerometer' to 'gravity' in the OSC prefix of the third cell of this notebook.

In [3]:
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn import *
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as ms
ms.use('seaborn-muted')
from IPython.display import clear_output, display

### 1. Importing the data, creating target values and training the regressor

In [4]:
#### read dataset
dataset = pd.read_csv('./data/examples10/postures.csv')

#importing the columns with accelerometer (gravity) data on the three exis
rawdata = dataset[['grx', 'gry', 'grz']].to_numpy()
inputs = np.empty((0,3))
target = np.empty((inputs.shape[0],2))

#iterating through the entries of the dataset and creating associated target values
#the index edges for the postures have been found manually by visually inspecting the waveforms
for i in range(0,rawdata.shape[0]):
    if (860 <= i <= 1000) or (1315 <= i <= 1450):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,1.]]), axis=0) #arm up
    elif (1750 <= i <= 1865) or (2130 <= i <= 2250):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,0.]]), axis=0) #arm side face up
    elif (2530 <= i <= 2650) or (2930 <= i <= 3055):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,1.]]), axis=0) #arm side face down
    elif (3490 <= i <= 3640) or (4150 <= i <= 4320):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,0.]]), axis=0) #arm down

#creating train/test split
inputs_train, inputs_test, target_train, target_test = sklearn.model_selection.train_test_split(inputs, target, test_size=0.1)

#training the model
mlp = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(8,4), max_iter=20000, activation='logistic')
mlp.fit(inputs_train, target_train)
target_predict =  mlp.predict(inputs_test)


#print the number of misclassified samples, accuracy and complete report (using scikit learn metric tools) 
print('r2 score on individual targets',sklearn.metrics.r2_score(target_test, target_predict, multioutput='raw_values'))

r2 score on individual targets [0.98823641 0.98502608]


### 2. Using the regressor with OSC real-time data

Here we use the regressor we previously trained. For a standalone application it would be more suitable to load the mlp model from a file (i.e. perform the training once for all and store it in a file).

The following cell will run until you press the stop button. However, before you can run it again you need to close the OSC server running the following cell.

In [ ]:
from pythonosc import dispatcher
from pythonosc import osc_server

acc_vect = np.zeros((1,3))

#creating a function that will handle and push accelerometer through the regressor
def acceleration_vector(address, args):
    if address.find('accelerometer/x') != -1:
        acc_vect[0,0] = args
    elif address.find('accelerometer/y') != -1:
        acc_vect[0,1] = args
    elif address.find('accelerometer/z') != -1:
        acc_vect[0,2] = args
        clear_output(wait=True)
        pred = mlp.predict(acc_vect)
        print('Parameters   %.3f'%pred.flat[0], '  %.3f' %pred.flat[1])

#attaching the function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/accelerometer/*", acceleration_vector)

#starting the OSC server
server = osc_server.ThreadingOSCUDPServer(('0.0.0.0', 8001), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
#closing the OSC server
server.server_close()

### 3. Follou-up activity

1. Create your dataset and use it to train the system.
2. Try to change the target values and include an additional third control parameter.


### 3. Follou-up assignment
Build a wekinator-like system, including the integration of an arbitrary an input device/source/app and an arbitrary controlled sound making device/target/app. The core (ML) of the system must be coded in python, the rest can be done in any language/environment. In groups, document your system design and implementation in a blog post, including a demo video (deadline in 2 weeks).